# Introduction

_Generative AI_ is a branch of deep learning that features the generation of samples from learned models that approximate probability distributions of interest.  For example, a generative model may be trained to represent the features of a specific movie actor.  Samples drawn from such a model would produce images that resemble the actor.  Generative models could also be trained to represent conditional probabilities, so that the sample generated is conditioned on an input of interest.  For example, a generative model could be trained to represent the features of many movie actors, but conditioned based on an input "prompt" so that the sample is restricted according to the prompt.  The prompt might be the name of the actor;  samples would then be generated to resemble that specific actor.  In many applications, such generation of sample images is an end in itself.  For example, a human user may use a Generative AI system to generate candidate images that represent a concept or graphic design, and could then select the most appealing one to be included on a web page or other document.  Other applications use Generative AI for "inference";  to find an answer to a specific query.  For example, in a robotic motion planning application, a Generative AI model could accept a prompt that represents a task goal and constraints, and could then generate a sample "answer", which would be a motion plan that achieves the task.  



The concepts of likelihood, log likelihood, and maximizing log likelihood are important foundations for algorithms used in generative AI.  These concepts are covered here first.  This is followed by a discussion of key concepts for understanding score-based models, which is an exciting new approach for learning complex probability distributions.

See, also


https://www.statlect.com/glossary/log-likelihood


https://github.com/jonkrohn/ML-foundations/blob/master/notebooks/5-probability.ipynb


Overall,
https://github.com/jonkrohn/ML-foundations/tree/master
is an excellent resource


These foundation concepts are important for understanding:
https://yang-song.net/blog/2021/score/

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

import math


In [ ]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

# This doesn't seem to fix equation numbering.

In [ ]:
%%javascript
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);

In [ ]:
def gaussian(mu, sigma, range, res):
    x = np.array([])
    y = np.array([])
    for x1 in np.arange(mu - range/2, mu + range/2, res):
        y_for_x = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x1 - mu)**2 / (2 * sigma**2) )
        x = np.append(x, x1)
        y = np.append(y, y_for_x)

    return x, y

def log_of_gaussian(mu, sigma, range, res):
    x = np.array([])
    y = np.array([])
    for x1 in np.arange(mu - range/2, mu + range/2, res):

        y_for_x = -np.log(sigma) - 0.5 * np.log(2 * np.pi) - (x1 - mu)**2 / (2 * sigma**2)

        x = np.append(x, x1)
        y = np.append(y, y_for_x)

    return x, y



# Likelihood

Let $\xi$ be a sample of observed data consisting of a set of datapoints:  

$$
\begin{equation}

\xi = [x_1, x_2 ... x_n]

\end{equation}
$$

Let $p(x_i)$ be a probability distribution for a datapoint, $x_i$, in $\xi$.
In ML models, such distributions are parameterized according to a parameter vector, $\theta$, so the distribution is written as $p(\theta, x_i)$.

The likelihood, $L(\theta, \xi)$ is the joint probability of all the datapoints in the sample:

$$
\begin{equation}

L(\theta, \xi) = \prod_{i = 1}^{n} p(\theta, x_i)

\end{equation}
$$

The log likelihood is then

$$
\begin{equation}

\ln L(\theta, \xi) = \ln \prod_{i = 1}^{n} p(\theta, x_i) = \sum_{i = 1}^{n} \ln p(\theta, x_i)

\end{equation}
$$


The log likelihood function is typically used to compute the \textit{maximum likelihood estimator} for a sample.  This is the value of $\theta$ that maximizes the log likelihood:

$$
\begin{equation}

\hat{\theta} = \argmax_{\theta} \ln L(\theta, \xi)

\end{equation}
$$

The log frequently used in maximum likelihood computations because it converts the product of distributions into a sum.  This is convenient for two reasons:  1) the asymptotic properties of sums are easier to analyze; and 2) sums are more numerically stable.

Consider, for example, the case where $p(\theta, x_i)$ is Gaussian (normal):

$$
\begin{equation}

p(\theta, x_i) = \left( 2 \pi \sigma^2 \right) ^ {-1/2} \exp \left( - \frac{1}{2} \frac{\left(x_i - \mu \right)^2}{\sigma^2} \right)

\end{equation}
$$

In this case, $\theta = \left[ \mu \quad \sigma^2 \right]$.

In [ ]:
x1, y1 = gaussian(0, 1.0, 10.0, 0.1)
plt.plot(x1, y1)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

For this distribution,

$$
\begin{equation}

\ln p(\theta, x_i) = \ln \left( \left( 2 \pi \sigma^2 \right) ^ {-1/2} \exp \left( - \frac{1}{2} \frac{\left(x_i - \mu \right)^2}{\sigma^2} \right) \right) = \ln \left( 2 \pi \sigma^2 \right) ^ {-1/2} + \ln \left( \exp \left( - \frac{1}{2} \frac{\left(x_i - \mu \right)^2}{\sigma^2} \right) \right) = - \ln ( \sigma ) - \frac{1}{2} \ln (2 \pi) - \frac{(x - \mu)^2}{2 \sigma ^2}

\end{equation}
$$

(see, also, https://stats.stackexchange.com/questions/404191/what-is-the-log-of-the-pdf-for-a-normal-distribution)

In [ ]:
x1, y1 = log_of_gaussian(0, 1.0, 10.0, 0.1)
plt.plot(x1, y1)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Note that this is a parabola, with it's peak (maximum point) at $\mu$.  Therefore, if a sample $\xi$, had only one datapoint, then the argmax operator for computing the maximum likelihood would set $\mu$ to the value of that datapoint.  Note, also, that the gradients of the parabola are lines that point to the optimum.  Thus, a numerical algorithm for maximizing $\mu$ that uses these derivatives would converge to the maximum. 

# Generative Modeling by Estimating Gradients of the Data Distribution

See https://yang-song.net/blog/2021/score/

Many generative modeling approaches, like Variational Auto Encoders, learn probability distributions via (approximate) maximum likelihood.  They then draw samples from these distributions (the generative aspect).  However, learning probability distributions directly can be challenging because the probability, $p_{\theta}(x)$, must be normalized (it must integrate to 1).  This is often computationally difficult.  (Note that $p_{\theta}(x)$ is the same as $p(\theta, \xi)$ in the above discussion).

Generally, the probability is of the form

$$
\begin{equation}

p_{\theta}(x) = \frac{e^{-f_\theta(x)}}{Z_\theta}

\end{equation}
$$

Note that with this general form, a Gaussian probability distribution is represented using $ Z_\theta = \left( 2 \pi \sigma^2 \right) $, and $ f_\theta(x) = \left( - \frac{1}{2} \frac{\left(x_i - \mu \right)^2}{\sigma^2} \right)$, ($\mu$ and $\sigma$ are elements of $\theta$).

For general probability distributions (but not Gaussian ones), computing $Z_\theta$ is often difficult.  An alternative that circumvents this problem is to model _the gradient of the log probability density function_.  This quantity is known as the (Stein) score function.  Such __score-based models__ are not required to have a tractable normalizing constant.  They can be directly learned using an approach called __score matching__.

The score function is defined as $ \nabla_x \ln p(x) $, and the score-based model, $s_\theta(x)$, is learned so that

$$
\begin{equation}

s_\theta(x) \approx \nabla_x \ln p(x) 

\end{equation}
$$

For the above general probability distribution

$$
\begin{equation}

 \ln p_\theta(x) = \ln \left( \exp \left( -f_\theta(x) \right) \right) - \ln \left(  Z_\theta \right) = -f_\theta(x) - \ln \left(  Z_\theta \right)

\end{equation}
$$

(the log of a fraction is the log of the numerator - the log of the denominator).  Taking the gradient yields

$$
\begin{equation}

 \nabla_x \ln p_\theta(x) = -\nabla_x \left( f_\theta(x) \right) - \nabla_x \ln \left(  Z_\theta \right) = -\nabla_x \left( f_\theta(x) \right)

\end{equation}
$$

Note that this does not depend on a normalizing factor; the $Z_\theta$ term has been removed.
For the Gaussian probability distribution, 


$$
\begin{equation}

 \ln p_\theta(x) = \ln \left( \exp \left( - \frac{1}{2} \frac{\left(x_i - \mu \right)^2}{\sigma^2} \right) \right) - \ln \left(  2 \pi \sigma^2 \right) = \left( - \frac{1}{2} \frac{\left(x_i - \mu \right)^2}{\sigma^2} \right) - \ln \left(  2 \pi \sigma^2 \right) 

\end{equation}
$$

and

$$
\begin{equation}

 \nabla_x \ln p_\theta(x) = \nabla_x \left( - \frac{1}{2} \frac{\left(x_i - \mu \right)^2}{\sigma^2} \right) - \nabla_x \ln \left(  2 \pi \sigma^2 \right) = \nabla_x \left( - \frac{1}{2} \frac{\left(x_i - \mu \right)^2}{\sigma^2} \right) = - \frac{\left(x_i - \mu \right)}{\sigma^2}

\end{equation}
$$

This is the (exact) __score function__ of the Gaussian probability distribution.

## Langevin dynamics for generating samples using a score function

An iterative procedure, called Langevin dynamics, can be used to draw samples of a distribution using the score function, rather than the distribution directly.
As mentioned before, this achieves the goal of generating samples, but without requiring an explicit representation of the distribution, and therefore, without the troublesome normalizing factor.
The procedure begins by computing an initial sample value $\mathbf{x}_0$ using an arbitrary distribution, like a uniform distribution, or a Gaussian.
Each iteration then proceeds according to the following equation:

$$
\begin{equation}

 \mathbf{x}_{i+1} \leftarrow \mathbf{x}_i + \epsilon \nabla_x \ln p_\theta(x) + \sqrt{2 \epsilon} \mathbf{z}_i, \quad i = 0, 1, ..., K

\end{equation}
$$

where $\mathbf{z}_i \sim \mathcal{N} (\mathbf{0}, \mathbf{I})$.  This converges to a sample from $p_(\mathbf{x})$ as $\epsilon \rightarrow 0, K \rightarrow \infty$.  In practice, the sample error is small when $\epsilon$ is sufficiently small, and $K$ is sufficiently large.

For example, consider the simple case of a Gaussian probability distribution with $\mu = 1$ and $\sigma = 0.5$.  Suppose we start with $\mathbf{x}_0$ arbitrarily set to -10.  The following cell shows how sampling converges.




In [ ]:
def langevin_dynamics_gaussian(x_init, eps, K, mu, sigma):
    # Consider generalizing this by passing in the score function as a parameter

    x_traj = np.array([])
    x = x_init
    for idx in range(0, K):
        # print("x: ", x)
        x_traj = np.append(x_traj, x)
        z = np.random.normal()
        x = x + eps * (- (x - mu) / sigma ** 2) + math.sqrt(2 * eps) * z

    plt.plot(x_traj)
    plt.xlabel("sample index")
    plt.ylabel("x")
    plt.show()

langevin_dynamics_gaussian(-5, 0.005, 3000, 1.0, 0.5)

# This doesn't seem to converge very well, even with very small values of eps, and very large values of K.
# Maybe the point is not to converge to mu, but rather, to behave in a way such that the samples look like they were
# drawn from the distribution defined by mu and sigma.  The latter seems to be the case.  With this perspective,
# convergence to the right behavior is fast.  However, it would be good to know when this convergence has been achieved.



## Training score-based models

Score-based models are trained by minimizing the __Fisher divergence__ between the model and the distribution.  The Fisher divergence is defined as

$$
\begin{equation}

\mathbb{E} \left[ \|  \nabla_x \ln p(x) - s_\theta(x) \| _2^2 \right]  

\end{equation}
$$

This cannot be used directly because it requires knowledge of $p(x)$.  Instead __score matching__ methods are used.  These minimize the Fisher divergence without requiring direct knowledge of $p(x)$.  Score matching objectives can directly be estimated on a dataset and optimized with stochastic gradient descent, analogous to the log-likelihood objective for training likelihood-based models (with known normalizing constants).

A significant challenge with basic score matching methods is that they result in models that are inaccurate in the (possibly large) regions where there is little or no training data (long tail problem).  (See https://yang-song.net/blog/2021/score/ for more detailed explanation).  If the regions of inaccuracy are large, then it is likely that the Langevin dynamics sampling process will start in such an inaccurate region, and will therefore derail the process.

A solution to this problem is to introduce noise perturbations, similar to diffusion policy models, and analogous to simulated annealing methods used for optimization.  The perturbed data points are used to train the score-based models.  When the noise magnitude is sufficiently large, it can populate low data density regions to improve the accuracy of estimated scores.
[Need to understand this better, intuitively.]

An important question is how much noise to add for the perturbation process.  Larger noise covers more low density regions for better score estimation, but it over-corrupts the data from the original distribution.  Smaller noise causes less corruption, but doesn't cover the low density regions as well.  To achieve the best of both worlds, multiple scales of noise perturbations are used simultaneously.   Each scale of perturbation is a Gaussian distribution with zero mean, and with variance determined by the scale index.  Thus, the procedure for generating a noisy training data point is as follows:

1.  Sample a data point $x \sim p(x)$
2.  The perturbed sample is then computed as $x_p = x + \sigma_i z$, where $z \sim \mathcal{N}(0, I)$, $i$ indicates the scale index, and $\sigma_i$ is the variance for the scale.

The next step is to train a set of __Noise Conditional Score-based Models__, $s_\theta(x, i)$ one for each noise scale. 

In order to gain intuition about how noising and de-noising can be used beneficially, we will use a 1D _Gaussian Mixture Model_ as an example probability distribution.  Consider such a model with a mixture of two sharp Gaussians, with a large, low-probability region in between the peaks.  Langevin sampling will fail if the initial sample is in the low-probability region;  there won't be enough information in the model to attract the Langevin sampling process to one of the two Gaussians.  The solution is to "soften" the Gaussians with noise, so that the region of attraction is larger.  The noise is added in multiple scales, as described above, so that there is a succession of models with increasing noise levels.  The de-noising process starts by sampling from the noisiest model in the scale, because this is the "softest".  The model is so soft that there are no "empty" regions;  any initial sample will fall into a region where there is gradient information that will move the Langevin sampling process in a productive direction.  When the Langevin sampling process is finished for the highest noise model, the overall process is continued with successively less noisy models.  A the end, the model with no noise is used, but here, the initial sample used will be the result of the previous process, which has "focused" the samples into known regions.  This overall process is analogous to _simulated annealing_, a popular technique for nonlinear optimization.

The following example code illustrates these concepts.


In [ ]:
# Gaussian mixture model
# Scikit has this, but has a lot of complexity, so a simple GMM capability is provided here.
# Note that this is limited to 1-d data.
# See, also, https://brilliant.org/wiki/gaussian-mixture-model/#:~:text=Gaussian%20mixture%20models%20are%20a,to%20learn%20the%20subpopulations%20automatically.

class GaussianMixtureModel:
    
    def __init__(self, mu_array, sigma_array, weight_array, range_min, range_max, res):

        # It is assumed that mu_array, sigma_array, and weight_array are the same length.  The length is the number of Gaussian components in the mixture model.
        # It is assumed that the elements of weight_array sum to 1.

        self.mu_array = mu_array
        self.sigma_array = sigma_array
        self.weight_array = weight_array
        self.range_min = range_min  # for plotting
        self.range_max = range_max  # for plotting
        self.res = res  # for plotting

    def evaluate(self, x1):

            # Compute distribution value (y) for random variable value x1.
            y_cum = 0
            for idx in range(len(self.mu_array)):
                y_for_idx = self.weight_array[idx] * 1/(self.sigma_array[idx] * np.sqrt(2 * np.pi)) * np.exp( - (x1 - self.mu_array[idx])**2 / (2 * self.sigma_array[idx]**2) )
                y_cum += y_for_idx

            return y_cum
    
    def numerical_gradient(self, x1, eps):

        # Compute the numerical derivative using x1 and x1 + eps
        y1 = self.evaluate(x1)
        y2 = self.evaluate(x1 + eps)
        return (y2 - y1) / eps

    def plot(self):

        # Arrays for plotting x-y values.
        x = np.array([])
        y = np.array([])

        for x1 in np.arange(self.range_min, self.range_max, self.res):

            # Compute distribution value (y) for random variable value x1.
            y_cum = self.evaluate(x1)

            # Append x1, y_cum to array for plotting.
            x = np.append(x, x1)
            y = np.append(y, y_cum)

        plt.plot(x, y)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.show()


    def sample(self, num_samples):

        # A general sampling technique, for any probability distribution, is inverse transform sampling.
        # See https://stats.stackexchange.com/questions/307686/generating-data-from-arbitrary-distribution
        # For Gaussian mixture models, there is a much easier way.
        # See https://stats.stackexchange.com/questions/243392/generate-sample-data-from-gaussian-mixture-model#:~:text=Sampling%20from%20mixture%20distribution%20is,%CE%BCI%20and%20%CF%83I.
        # This method is used here.

        # First sample "category" (component)
        category_samples = np.random.choice(len(self.weight_array), size=num_samples, p=self.weight_array)

        # Next, sample from the Gaussian component.
        samples = np.array([])
        for idx in range(len(category_samples)):
            component_sample = np.random.normal(self.mu_array[category_samples[idx]], self.sigma_array[category_samples[idx]], 1)
            samples = np.append(samples, component_sample)

        return samples
    

    def fake_langevin_dynamics(self, x_init, eps, K, print_details_p):
        
        # Simplified langevin dynamics, using numerical derivatives rather than score function

        delta = 0.01  # delta (or eps) used for numerical differentiation

        x_traj = np.array([])
        x = x_init
        for idx in range(0, K):
            x_traj = np.append(x_traj, x)
            z = np.random.normal()
            dydx = self.numerical_gradient(x, delta)

            if print_details_p:
                print("x: ", x)
                print("dydx: ", dydx)

            x = x + eps * dydx + math.sqrt(2 * eps) * z

        return x_traj


    def convolve_with_gaussian(self, mu, sigma):

        # Returns a new gmm with the convolved parameters

        new_mu_array = np.array([])
        new_sigma_array = np.array([])

        for idx in range(len(self.mu_array)):
            new_mu_array = np.append(new_mu_array, self.mu_array[idx] + mu)
            new_sigma_array = np.append(new_sigma_array, np.sqrt(self.sigma_array[idx]**2 + sigma**2))
            # See http://www.lucamartino.altervista.org/2003-003.pdf

        convolved_gmm = GaussianMixtureModel(new_mu_array, new_sigma_array, self.weight_array, self.range_min, self.range_max, self.res)
        return convolved_gmm




## Intuitive understanding of de-noising

Consider the following "ground truth" model, consisting of two peaks in a Gaussian mixture model.  This is similar to the diagrams in Song's blog, but is even simpler in order to illustrate the key point of how noise helps.

In [ ]:
# Example GMM

gmm1 = GaussianMixtureModel(np.array([-5, 5]), np.array([0.2, 0.5]), np.array([0.2, 0.8]), -8.0, 8.0, 0.1)
gmm1.plot()

If a Langevin dynamics sampling process were started from such a (non-noised model), the initial sample would likely fall into a low-probability region.  This would derail the Langevin dynamics.

This failure is shown in detail in the following code.  Unfortunately, while it is easy to analytically derive a score-based model for a Gaussian probability distribution (see above computations and subsequent Langevin dynamics sampling), it is difficult to do this for a GMM.  Therefore, the score-based model would have to be trained (for real) using the score-matching training algorithm.  This is beyond the scope of these notes, and is not necessary for gaining an intuitive understanding of de-noising.  Instead, we will sample from the GMMs directly, and will see what happens to samples as they pass through successive, less noisy layers.  This illustrates the "annealing" process.  A simplified "fake" Langevin dynamics is used to illustrate this process.

In [ ]:
langevin_traj_1 = gmm1.fake_langevin_dynamics(0.0, 0.01, 10, True)

As can be seen, the gradients for the initial sample (in this case, 0) are very small.  This can cause the Langevin dynamics process to become "stuck" in a low-probability region, wandering endlessly, with no good notion of a direction to go in.

Now, let's see how this situation can be improved by introducing a noise schedule.

In [ ]:
# Define noise schedule

sigma_1 = 1.0
sigma_2 = 2.0
sigma_3 = 4.0

gmm_noise_1 = gmm1.convolve_with_gaussian(0, sigma_1)
gmm_noise_1.plot()

gmm_noise_2 = gmm1.convolve_with_gaussian(0, sigma_2)
gmm_noise_2.plot()

gmm_noise_3 = gmm1.convolve_with_gaussian(0, sigma_3)
gmm_noise_3.plot()

As can be seen from these plots, in the noisiest model, there are no longer any regions of low probability.  The Langevin dynamics process now works better because the gradients are not close to 0.

In [ ]:
langevin_traj_4 = gmm_noise_3.fake_langevin_dynamics(0.0, 0.01, 10, True)

The de-noising process then proceeds as follows.

In [ ]:
langevin_traj_4 = gmm_noise_3.fake_langevin_dynamics(0.0, 0.01, 100, False)   # Same as above, but more iterations

plt.plot(langevin_traj_4)
plt.xlabel("sample index")
plt.ylabel("x")
plt.show()

noise_level_3_result = langevin_traj_4[-1]
print("noise level 3 result: ", noise_level_3_result)

langevin_traj_3 = gmm_noise_2.fake_langevin_dynamics(noise_level_3_result, 0.01, 100, False)  # Result from previous noise level is used to initialize this

plt.plot(langevin_traj_3)
plt.xlabel("sample index")
plt.ylabel("x")
plt.show()

noise_level_2_result = langevin_traj_3[-1]
print("noise level 2 result: ", noise_level_2_result)

langevin_traj_2 = gmm_noise_1.fake_langevin_dynamics(noise_level_2_result, 0.01, 100, False)

plt.plot(langevin_traj_2)
plt.xlabel("sample index")
plt.ylabel("x")
plt.show()

noise_level_1_result = langevin_traj_2[-1]
print("noise level 1 result: ", noise_level_1_result)

langevin_traj_final = gmm1.fake_langevin_dynamics(noise_level_1_result, 0.01, 100, False)

plt.plot(langevin_traj_final)
plt.xlabel("sample index")
plt.ylabel("x")
plt.show()

noise_level_0_result = langevin_traj_final[-1]
print("noise level 0 result: ", noise_level_0_result)



In [ ]:
# Miscellaneous

# Non-Langevin sampling;  not really relevant here.

# samples_1 = gmm_noise_1.sample(10)
# print("samples_1", samples_1)

# samples_2 = gmm_noise_2.sample(10)
# print("samples_2", samples_2)

# samples_3 = gmm_noise_3.sample(100)
# print("samples_3", samples_3)









To do:  provide conclusion, finish code below.

A complete implementation of the de-noising process for the above Gaussian mixture model noise schedule would require computing the score-based models, and then using Langevin dynamics to sample.  Unfortunately, while it is easy to analytically derive a score-based model for a Gaussian probability distribution (see above computations and subsequent Langevin dynamics sampling), it is difficult to do this for a GMM.  Therefore, the score-based model would have to be trained (for real) using the score-matching training algorithm.  This is beyond the scope of these notes, and is not necessary for gaining an intuitive understanding of de-noising.  Instead, we will sample from the GMMs directly, and will see what happens to samples as they pass through successive, less noisy layers.  This illustrates the "annealing" process.  A simplified Langevin dynamics is needed to illustrate this process.




In [ ]:
class GmmDenoiser:

    def __init__(self, gmm_original, noise_schedule):

        # noise_schedule is assumed to be an array of monotonically increasing sigma values

        self.noise_models = np.array([])

        for sigma in noise_schedule:
            gmm_with_noise = gmm_original.convolve_with_gaussian(0, sigma)
            self.noise_models = np.append(self.noise_models, gmm_with_noise)

        self.gmm_original = gmm_original

    def generate_sample(self):

        for gmm_with_noise in self.noise_models:
            
        